In [1]:
#IPython is what you are using now to run the notebook
import IPython
print( "IPython version:      %6.6s (need at least 1.0)" % IPython.__version__)

# Numpy is a library for working with arrays and matrices
import numpy as np
print( "Numpy version:        %6.6s (need at least 1.7.1)" % np.__version__)

# SciPy implements many different numerical algorithms
import scipy as sp
print( "SciPy version:        %6.6s (need at least 0.12.0)" % sp.__version__)

# Pandas makes working with data tables easier
import pandas as pd
print( "Pandas version:       %6.6s (need at least 0.11.0)" % pd.__version__)

# Module for plotting
import matplotlib.pyplot as plt  
from pylab import *
print( "Mapltolib version:    %6.6s (need at least 1.2.1)" %
       matplotlib.__version__)
%matplotlib inline
# necessary for in-line graphics

# SciKit Learn implements several Machine Learning algorithms
import sklearn
print( "Scikit-Learn version: %6.6s (need at least 0.13.1)" %
       sklearn.__version__)
import os
# for certain system-related functions

IPython version:       7.8.0 (need at least 1.0)
Numpy version:        1.16.5 (need at least 1.7.1)
SciPy version:         1.3.1 (need at least 0.12.0)
Pandas version:       0.25.1 (need at least 0.11.0)
Mapltolib version:     3.1.1 (need at least 1.2.1)
Scikit-Learn version: 0.21.3 (need at least 0.13.1)


# 1 Descriptive analysis

  ## 1.1 Summary statistics

First, learn about data.

1. Report summary statistics (mean, standard deviation, and number of missings) for all of the demographic variables in the dataset (i.e., everything except year, folnum, village). A central variable, progresa is coded in a rather unintuitive way. Find it's actual coding scheme. Does this fit with the documentation above?

Present these in a single table alphabetized by variable name. Do NOT simply expect the grader to scroll through your output!

Response : 

The below table displays the summary statistics for all the demographic variables in the dataset.

The "progresa" variable is actually used to identify whether the observations belong to the "treatment" or "control" group. The "basal" value corresponds to the treatment group and the "0" value corresponds to the control group. This variable is central to our analysis according to the case documentation and is heavily used in our analysis to establish causality using estimators.

In [2]:
#Reading the progresa data
progress_df= pd.read_csv('C:/Users/vaide/Documents/574-DS2/ps3/progresa-sample.csv.bz2')
progress_df.head()

#progress_df['progresa'] = progress_df['progresa'].map({'basal' : '1', '0':'0'})
#progress_df['poor'] = progress_df['poor'].map({'pobre' : '1', 'no pobre':'0'})

# Getting data except year , flnum and village
progresa_req_df = progress_df.drop(['year', 'folnum', 'village'], axis = 1)

# Getting only mean and standard deviation from the describe function
prog_sum_stats=pd.DataFrame(progresa_req_df.describe()['mean':'std'].T.sort_index())

# Calculating the number of missing values
prog_sum_stats['Number of Missing Values']=progresa_req_df.isnull().sum()

# Naming the columns
prog_sum_stats.columns=["Mean","Standard Deviation","Number of Missing Values"]

# Printing the summary statistics
prog_sum_stats

,Mean,Standard Deviation,Number of Missing Values
age,11.366460,3.167744,0
dist_cap,147.674452,76.063134,0
dist_sec,2.418910,2.234109,0
fam_n,7.215715,2.352900,0
grc,3.963537,2.499063,6549
grc97,3.705372,2.572387,0
hohage,44.436717,11.620372,10
hohedu,2.768104,2.656106,0
hohsex,0.925185,0.263095,20
hohwag,586.985312,788.133664,0


## 1.2 Differences at baseline?

Now let's investigate the differences in baseline. Are the baseline (1997) demographic characteristics for
the poor different in treatment and control villages?

1. Use t-test to determine whether there is a statistically significant difference in the average values of
each of the variables in the dataset. Focus only on the data from 1997 for poor.

2. Present your results in a single table with the following columns and 14 (or so) rows:

Variable Avg (Treatment villages) Avg (control) difference p-value

sex         0.5193                  0.5051        0.0143     0.0122

3. Are there statistically significant differences between treatment and control villages as baseline?
4. Why does it matter if there are differences at baseline?
5. What does this imply about how to measure the impact of the treatment?

Response:

1. We have filtered the data and performed a t-test on all observations corresponding to the baseline (1997) demographic characteristics for the poor population.


2. The p-value obtained from the t-test is appended to our results table for all the 15 variables.


3. From the below table, we can say that there is statistical significance between treatment and control villages as baseline as the p-value is less than 0.05. Thus, we can say that there is significant difference for sex, dist_sec (nearest distance to a secondary school), min_dist (min distance to an urban center), dist_cap (min distance to the capital), hohedu (years of schooling of head of household), hohwag (monthly wages of head of household),hohage (age of head of household) and the welfare_index (welfare index used to classify poor) variables.


4. It matters if there are differences at baseline because if we want to do randomization in exploring the causal relationships between people and treatment and if the treatment and control groups had a difference at baseline, the differences shown after the treatment may not be valid, because we can not prove that the changes happend to the subjects are caused by the treatment directly. Thus, for any comparison between treatment and control, it is required to have a same baseline so as to gauge the actual effect of the treatment.


5. In this study, we want to measure the impact of the treatment by calculating the differences between treatment and control groups. From the table, we can see that there is existing difference for some variables in the treatment and control groups and hence it would be difficult to measure the true impact of the treatment. In this case, other strategies such as stratified sample, double differences etc. can be used to to measure the impact of treatment without bias.

In [3]:
#Filtering 
progresa_filtered = progress_df[(progress_df.poor=='pobre') & (progress_df.year==97)] 

#Created new df by removing year, folnum, village columns
progresa_df_edited = progresa_filtered.drop(['year','folnum','village'],axis=1)

#Calculating mean values for treatment and control set
progresa_treatment_control = pd.DataFrame(pd.merge(progresa_df_edited[progresa_df_edited.progresa == 'basal'].mean().reset_index(name='Average value (Treatment villages)'), progresa_df_edited[progresa_df_edited.progresa == '0'].mean().reset_index(name='Average value (Control villages)'),on=['index']))

progresa_treatment_control

# Creating differences list to store the difference of average for treatement and contol villages
differences = []
for i in range(len(progresa_treatment_control)):
    differences.append(round((progresa_treatment_control['Average value (Treatment villages)'][i] - progresa_treatment_control['Average value (Control villages)'][i]), 5))

# Creating p-value list to store the p-value for all columns
p_value = []
# Creating a list to store the variable is statistically significant
stats_significant = []


from scipy.stats import stats
#Iterating over the df to calculate t, p value and statistical significancy
for i in list(progresa_treatment_control['index']):
          p_value.append(stats.ttest_ind(progresa_df_edited[progresa_df_edited.progresa == 'basal'][i], progresa_df_edited[progresa_df_edited.progresa == '0'][i], nan_policy='omit').pvalue)
          if stats.ttest_ind(progresa_df_edited[progresa_df_edited.progresa == 'basal'][i], progresa_df_edited[progresa_df_edited.progresa == '0'][i], nan_policy='omit').pvalue < 0.05:
                stats_significant.append('TRUE')
          else: 
            stats_significant.append('FALSE')
            
# Using the above created lists to create 3 columns as needed        
progresa_treatment_control['Difference (Treat - Control)'] = differences   
progresa_treatment_control['p-value'] = p_value
progresa_treatment_control['Statistically_Significant'] = stats_significant 

# Naming the columns
progresa_treatment_control.columns=["Variable", "Avg (Treatment Villages)" ,"Avg (Control Villages)","Difference","p-value","Stat Sig"]

# Printing the required table
progresa_treatment_control

,Variable,Avg (Treatment Villages),Avg (Control Villages),Difference,p-value,Stat Sig
0,sex,0.519317,0.505052,0.01427,1.219172e-02,TRUE
1,indig,0.325986,0.332207,-0.00622,2.453603e-01,FALSE
2,dist_sec,2.453122,2.507662,-0.05454,3.569843e-02,TRUE
3,sc,0.822697,0.815186,0.00751,9.517806e-02,FALSE
4,grc,3.531599,3.543050,-0.01145,6.890151e-01,FALSE
5,fam_n,7.281327,7.302469,-0.02114,4.271039e-01,FALSE
6,min_dist,107.152915,103.237854,3.91506,2.358312e-16,TRUE
7,dist_cap,150.829074,153.769730,-2.94066,8.415005e-04,TRUE
8,hohedu,2.663139,2.590348,0.07279,1.105093e-02,TRUE
9,hohwag,544.339544,573.163558,-28.82401,3.253835e-04,TRUE


## 2 Measuring Impact

Our goal is to estimate the causal impact of the Progresa program on the schooling outcomes of individuals
in Mexico. We will focus on the impact of the program on the poor, since only the poor were eligible to
receive the Progresa assistance.

## 2.1 Before-after estimator

First, implement the before-after estimator. Compare the schooling rate of poor households in progresa
villages before (i.e. 1997) and after (i.e. 1998) the program.


1. compute the estimator by just comparing the average schooling rates for these villages.

Response: 

From the below calculation, we can see that the average schooling rate of the poor population before the progresa program is lower than the average schooling rate after the progresa program.

In [4]:
# Creating a after column for estimating the before-after estimator
progress_df['after']=progress_df.year > 97
progress_df[progress_df.poor=="pobre"].groupby('after').sc.mean()

after
False    0.819837
True     0.831730
Name: sc, dtype: float64

2. now re-compute the estimator using linear regression, and individual schooling rates. Do not include
other regressors.

The below simple linear regression model below shows the interaction between the individual schooling rate for poor and the after variable. This model is statistically significant as the p-value of the predictor variable is < 0.05 and shows that the schooling rate was lower before the program was implemented i.e The schooling rate increased by 0.012 after the program was implemented and 


In [5]:
import statsmodels.formula.api as smf

m = smf.ols(formula = 'sc ~ after', data=progress_df[progress_df.poor =='pobre'])
r = m.fit()
r.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     14.28
Date:                Wed, 05 Feb 2020   Prob (F-statistic):           0.000158
Time:                        11:35:01   Log-Likelihood:                -26278.
No. Observations:               58372   AIC:                         5.256e+04
Df Residuals:                   58370   BIC:                         5.258e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.8198      0.002    379.824      0.000       0.816       0.824
after[T.True]     0.0119      0.003      3.779      0.000       0.006       0.018
==============================================================================
Omnibus:                    15383.245   Durbin-Watson:                   1.397
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30726.067
Skew:                          -1.714   Prob(JB):                         0.00
Kurtosis:                       3.939   Cond. No.                         2.55
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

3. Finally, estimate a multiple regression model that includes other covariates.

Response:

In the below multiple regression model, we can see that this model is different to previous model. Here we are considering all the regressors and many regressors are not statistically significant to the model.  Also, even after including other factors in the model, the individual schooling rate is not affected after the program was implemented in 1998 and it is still statistically significant as the p-vlaue < 0.05


In [6]:
mulmodel_ba=smf.ols(formula='sc~after+age+dist_cap+dist_sec+fam_n+grc+grc97+hohage+hohedu+hohsex+hohwag+indig+min_dist+sc+sc97+sex+welfare_index',data=progress_df[progress_df.poor=="pobre"])
mulfit_ba=mulmodel_ba.fit()
mulfit_ba.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.570e+31
Date:                Wed, 05 Feb 2020   Prob (F-statistic):               0.00
Time:                        11:35:02   Log-Likelihood:             1.8011e+06
No. Observations:               56893   AIC:                        -3.602e+06
Df Residuals:                   56875   BIC:                        -3.602e+06
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -2.425e-15    2.1e-16    -11.525      0.000   -2.84e-15   -2.01e-15
after[T.True] -2.177e-16   4.44e-17     -4.908      0.000   -3.05e-16   -1.31e-16
age           -1.396e-16   1.31e-17    -10.640      0.000   -1.65e-16   -1.14e-16
dist_cap      -1.198e-17   3.46e-19    -34.670      0.000   -1.27e-17   -1.13e-17
dist_sec         1.8e-16    8.7e-18     20.698      0.000    1.63e-16    1.97e-16
fam_n           4.63e-17   8.32e-18      5.564      0.000       3e-17    6.26e-17
grc            6.744e-16   3.51e-17     19.197      0.000    6.06e-16    7.43e-16
grc97         -3.931e-16   3.55e-17    -11.082      0.000   -4.63e-16   -3.24e-16
hohage         1.711e-16   1.85e-18     92.719      0.000    1.67e-16    1.75e-16
hohedu         1.908e-16   8.14e-18     23.451      0.000    1.75e-16    2.07e-16
hohsex         6.939e-18   7.17e-17      0.097      0.923   -1.34e-16    1.47e-16
hohwag         6.268e-19   2.62e-20     23.898      0.000    5.75e-19    6.78e-19
indig         -1.665e-16   4.46e-17     -3.733      0.000   -2.54e-16   -7.91e-17
min_dist      -7.047e-18   5.96e-19    -11.823      0.000   -8.22e-18   -5.88e-18
sc                1.0000   8.98e-17   1.11e+16      0.000       1.000       1.000
sc97          -1.457e-16   9.32e-17     -1.564      0.118   -3.28e-16    3.69e-17
sex           -5.551e-17   3.63e-17     -1.528      0.127   -1.27e-16    1.57e-17
welfare_index -1.217e-17   1.71e-19    -71.177      0.000   -1.25e-17   -1.18e-17
==============================================================================
Omnibus:                     1881.717   Durbin-Watson:                   0.128
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2214.131
Skew:                          -0.412   Prob(JB):                         0.00
Kurtosis:                       3.504   Cond. No.                     1.22e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.22e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

4. compare all the estimators. Are your estimates statistically significant? What do they suggest about
the efficacy of the progresa program?

Response:

By comparing the the average rate and the simple regression model, we can see that the individual schooling rates are higher after the program was implemented. But in multiple regressin model, we can see that even after all the regressors are added the schooling rate after the program was implemented is not affected. Thus, we can say that, our model is not so accurate as the before and after estimator is depends on only one parameter, without considering other factors.

## 2.2 Cross-sectional estimator

Now let's implement the cross-sectional estimator. Proceed along the same lines as what you did above.


1. Begin by estimating the impact of Progresa by compring the average enrollment rate among poor households in the treatment villages and the average enrollment rate among poor households in the control villages. What do you find?

Response: 

From the below calculation, we can see that the average schooling rate of the poor population of control villages is lower than the average schooling rate of treatment villages

In [7]:
progress_df[progress_df.poor =='pobre'].groupby('progresa').sc.mean()

progresa
0        0.811641
basal    0.833891
Name: sc, dtype: float64

2. Now repeat the estimator using simple regression.

Response:
    
The simple linear regression model below shows the interaction between the individual schooling rate for poor and the progessa  variable. We can see that this model is statistically significant as the p-value of the predictor variable is < 0.05 and shows that the schooling rate is affected positively by the treatment groups. ie treatment groups have an effect of (0.022) on the scolling rate.  

In [8]:
m = smf.ols(formula = 'sc ~ progresa', data=progress_df[progress_df.poor =='pobre'])
r = m.fit()
r.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     47.30
Date:                Wed, 05 Feb 2020   Prob (F-statistic):           6.16e-12
Time:                        11:35:02   Log-Likelihood:                -26261.
No. Observations:               58372   AIC:                         5.253e+04
Df Residuals:                   58370   BIC:                         5.254e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8116      0.003    318.680      0.000       0.807       0.817
progresa[T.basal]     0.0222      0.003      6.877      0.000       0.016       0.029
==============================================================================
Omnibus:                    15367.207   Durbin-Watson:                   1.394
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30673.992
Skew:                          -1.713   Prob(JB):                         0.00
Kurtosis:                       3.938   Cond. No.                         3.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

3. Third, use multiple regression to get the same estimate.

Response:

The multiple linear regression model below shows the interaction berween the individual schooling rate and all the regressors. We can see that this model is different from the previous model as many predictors are not statistically significant to the model. In this model, on considering all the other regressors , the treatment group is having effect on scholling rate and it is is statistically significant as the p-value < 0.05.


In [9]:
mulmodel_ba=smf.ols(formula='sc~progresa+age+dist_cap+dist_sec+fam_n+grc+grc97+hohage+hohedu+hohsex+hohwag+indig+min_dist+sc+sc97+sex+welfare_index',data=progress_df[progress_df.poor=="pobre"])
mulfit_ba=mulmodel_ba.fit()
mulfit_ba.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.869e+30
Date:                Wed, 05 Feb 2020   Prob (F-statistic):               0.00
Time:                        11:35:03   Log-Likelihood:             1.7265e+06
No. Observations:               56893   AIC:                        -3.453e+06
Df Residuals:                   56875   BIC:                        -3.453e+06
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept          -7.91e-16   7.85e-16     -1.007      0.314   -2.33e-15    7.48e-16
progresa[T.basal]  -1.21e-15   1.39e-16     -8.687      0.000   -1.48e-15   -9.37e-16
age               -4.601e-16   4.81e-17     -9.571      0.000   -5.54e-16   -3.66e-16
dist_cap           1.409e-18   1.28e-18      1.098      0.272   -1.11e-18    3.93e-18
dist_sec           2.507e-16   3.22e-17      7.775      0.000    1.87e-16    3.14e-16
fam_n              1.871e-16   3.09e-17      6.065      0.000    1.27e-16    2.48e-16
grc                9.673e-16   1.12e-16      8.631      0.000    7.48e-16    1.19e-15
grc97              4.207e-17   1.09e-16      0.385      0.701   -1.72e-16    2.56e-16
hohage             1.427e-16   6.84e-18     20.852      0.000    1.29e-16    1.56e-16
hohedu            -1.596e-16   3.02e-17     -5.290      0.000   -2.19e-16      -1e-16
hohsex            -3.123e-17   2.66e-16     -0.118      0.906   -5.52e-16     4.9e-16
hohwag            -2.033e-17   9.73e-20   -208.975      0.000   -2.05e-17   -2.01e-17
indig              2.463e-16   1.65e-16      1.489      0.136   -7.79e-17    5.71e-16
min_dist          -4.662e-18   2.22e-18     -2.102      0.036   -9.01e-18   -3.14e-19
sc                    1.0000   3.33e-16      3e+15      0.000       1.000       1.000
sc97               4.927e-16   3.45e-16      1.428      0.153   -1.83e-16    1.17e-15
sex               -3.261e-16   1.35e-16     -2.420      0.016    -5.9e-16    -6.2e-17
welfare_index      4.066e-20   6.34e-19      0.064      0.949    -1.2e-18    1.28e-18
==============================================================================
Omnibus:                    67148.158   Durbin-Watson:                   0.330
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         10829697.516
Skew:                           6.191   Prob(JB):                         0.00
Kurtosis:                      69.447   Cond. No.                     1.23e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.23e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

4. Finally, as above, compare your three estimators. What do you find? Are the effects statistically significant?

Response:

For this estimator , by comparing the average enrollment rate for poor population in treatement villages and in control villages and by simple regression , that the schooling rates for control villages was less than that of treatement villages. But in multiple resgressin model after adding all the variables, we can see that the schooling rate is hardly affected in the treatment villages group, which could mean that our model is not very accurate ad we could have ignored some unobserved characteristics and assumptions by considering only the treatment and control villages groups.  

### 2.3 Differences-in-differences estimator

Now we are ready for DiD estimator. Proceed along the same lines as above.

1. Start with the simple table. However, DiD requires 4-way comparison. So compare the average enrollment rate among poor households in the treatment villages and the average enrollment rate among poor households in the control villages, both 1997 and 1998. What do you find?

Response :

By considering the DiD estimator ,we can see from the below calculation that before the progresa program was implemented in 1997, the average enrollment rate among poor households in the treatment villages was higher than that of the control villages. After the progresa program was implemented in 1998, the average enrollment rate among poor households in the treatment villages was also higher than that of the control villages.

In [10]:
progress_df[progress_df.poor =='pobre'].groupby(['progresa','after']).sc.mean()

progresa  after
0         False    0.815186
          True     0.807637
basal     False    0.822697
          True     0.846479
Name: sc, dtype: float64

2. Now repeat the estimator using simple regression.

From the below simple regression model, we can see that the overall estimate of the treatment effect is 0.0313 after the program was implemented and it is statistically significant as the p-value < 0.05.

In [11]:
m = smf.ols(formula = 'sc ~ after*progresa', data=progress_df[progress_df.poor =='pobre'])
r = m.fit()
r.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     28.31
Date:                Wed, 05 Feb 2020   Prob (F-statistic):           2.76e-18
Time:                        11:35:04   Log-Likelihood:                -26242.
No. Observations:               58372   AIC:                         5.249e+04
Df Residuals:                   58368   BIC:                         5.253e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                           0.8152      0.003    233.182      0.000       0.808       0.822
after[T.True]                      -0.0075      0.005     -1.480      0.139      -0.018       0.002
progresa[T.basal]                   0.0075      0.004      1.691      0.091      -0.001       0.016
after[T.True]:progresa[T.basal]     0.0313      0.006      4.835      0.000       0.019       0.044
==============================================================================
Omnibus:                    15346.988   Durbin-Watson:                   1.397
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30608.651
Skew:                          -1.711   Prob(JB):                         0.00
Kurtosis:                       3.937   Cond. No.                         7.67
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

3. And as above, use multiple regression to get the same estimate.

In the below multiple regression model, we can see that this model is different from the previous model. Here we are considering all the regressors and many of these regressors are not statistically significant to the model. From this model , we can see that by considering all the other variables the the treatment villages group after the program has a slight positive impact on the schooling rate and is statistically significant as the p-value < 0.05.


In [12]:
mulmodel_ba=smf.ols(formula='sc~after*progresa+age+dist_cap+dist_sec+fam_n+grc+grc97+hohage+hohedu+hohsex+hohwag+indig+min_dist+sc+sc97+sex+welfare_index',data=progress_df[progress_df.poor=="pobre"])
mulfit_ba=mulmodel_ba.fit()
mulfit_ba.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 7.572e+30
Date:                Wed, 05 Feb 2020   Prob (F-statistic):               0.00
Time:                        11:35:04   Log-Likelihood:             1.7695e+06
No. Observations:               56893   AIC:                        -3.539e+06
Df Residuals:                   56873   BIC:                        -3.539e+06
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                       -9.194e-16   3.71e-16     -2.478      0.013   -1.65e-15   -1.92e-16
after[T.True]                   -2.729e-15   1.12e-16    -24.417      0.000   -2.95e-15   -2.51e-15
progresa[T.basal]               -6.896e-16   8.87e-17     -7.774      0.000   -8.63e-16   -5.16e-16
after[T.True]:progresa[T.basal]  3.252e-15   1.31e-16     24.882      0.000       3e-15    3.51e-15
age                             -2.446e-16   2.29e-17    -10.694      0.000   -2.89e-16      -2e-16
dist_cap                        -5.529e-18   6.03e-19     -9.163      0.000   -6.71e-18   -4.35e-18
dist_sec                         8.674e-17   1.52e-17      5.724      0.000     5.7e-17    1.16e-16
fam_n                            1.996e-16   1.45e-17     13.765      0.000    1.71e-16    2.28e-16
grc                              2.862e-16   6.12e-17      4.675      0.000    1.66e-16    4.06e-16
grc97                            7.416e-17   6.18e-17      1.199      0.230    -4.7e-17    1.95e-16
hohage                           1.156e-16   3.22e-18     35.935      0.000    1.09e-16    1.22e-16
hohedu                          -1.596e-16   1.42e-17    -11.255      0.000   -1.87e-16   -1.32e-16
hohsex                          -8.674e-17   1.25e-16     -0.695      0.487   -3.32e-16    1.58e-16
hohwag                           2.414e-18   4.57e-20     52.787      0.000    2.32e-18     2.5e-18
indig                            2.637e-16   7.77e-17      3.392      0.001    1.11e-16    4.16e-16
min_dist                          1.03e-17   1.04e-18      9.880      0.000    8.26e-18    1.23e-17
sc                                  1.0000   1.57e-16   6.38e+15      0.000       1.000       1.000
sc97                             2.776e-17   1.63e-16      0.171      0.864   -2.91e-16    3.46e-16
sex                             -1.908e-17   6.33e-17     -0.301      0.763   -1.43e-16    1.05e-16
welfare_index                    2.738e-18   2.98e-19      9.186      0.000    2.15e-18    3.32e-18
==============================================================================
Omnibus:                    29140.361   Durbin-Watson:                   0.090
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           590477.812
Skew:                          -2.010   Prob(JB):                         0.00
Kurtosis:                      18.262   Cond. No.                     1.24e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.24e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

4. Finally, as above, compare your three estimators. What do you find? Are the effects statistically significant?

Response:

For the DiD estimator , we are considering the effect of schooling rate on the treatment villages and control villages groups before and after the programwas implemented. By comparing the average enrollment rate for poor population in the treatment and control villages in year 1997 and 1998, we see that there is a positive effect in the schooling rate for treatment villages group after the program was implemented and this effect is statistically significant since p-value < 0.05 in all the cases.


## 2.4 Compare the estimators

Now you have used three estimators to assess the effect of Progresa program.




 ### 1. List the identifying assumptions (counterfactual assumptions) behind all three models. Which ones do you find more/less plausible?

Response: 

Model 1:

The counterfactual assumption behind the first model is that in the average enrollment change before and after the program is the same.

Model 2:

The counterfactual assumption behind the second model is that the average enrollment rates in the treatment villages and control villages are the same before and after the program.

Model 3:

The counterfactual assumption behind the third model is that without the progresa treatment, the change happens in the treatment group would be the same as the control group.

The first 2 assumptions seem less plausible as they do not consider the change in treatment and control groups before and after the program.

### 2. Compare the estimates of all three models. Do your analysis suggest that progresa program had a positive impact on schooling rates?

Response :

We have three kinds of estimators. 
1.) Before-after estimator
2.) Cross sectional estimator
3.) Difference in difference estimator. 

In the first estimator, we consider effect on the schooling rate, before and after the program was implemented. This is not quite accurate as the the difference before and after has been measured solely on the basis of one parameter. There could be other factors in the progresa villages which could attribute to the increase in schooling rate eg. increase in income, standard of living etc. which remain unknown due to this method.

In the second estimator we only considers the treatment and control goup effect on schooling rate in the progresa villages. Here too, we ignore unobserved characteristics and assumptions as we do not take into consideration the before and after effect of the implementation of the program.

According to me, difference in difference estimator is a better method as it eliminates any unobserved characteristics and it gets subtracted in the difference. The diff-in-diff estimator considers the effect on the schooling rate on the treatment and control villages before and after the program and seems to relax the assumption of conditional exogeneity or selection on observed characteristics.

Thus, according to my analysis, the Progresa program had a causal impact on the enrollment rates of the poor households in Mexico and the increase in schooling enrollment was statistically significant. However, to make a more accurate judgement, we need more economic details to ascertain the efficacy of this program.